In [41]:
#importing libraries
import pandas as pd
import re
import nltk

In [42]:
#load the dataset
df = pd.read_csv("tweet_emotions .csv")

In [43]:
df

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [44]:
#data Preprocessing
def preprocess_text(text):
    text=re.sub(r"[^a-zA-Z0-9]", " ", text)
    text=re.sub(r"http\S+|www\S+|@[^\s]+", "", text)
    text=text.lower()

    return text

In [45]:
df=df[['sentiment', 'content']]
df['content']=df['content'].apply(preprocess_text)

<ipython-input-45-83d235692f90>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].apply(preprocess_text)


In [46]:
#feature extraction
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
#using TF-IDFvectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(stop_words=stop_words)
X=vectorizer.fit_transform(df['content'])
y=df['sentiment']

In [48]:
#split the dataset into training&testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
#model building and evaluation using SVM
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [50]:
#evaluate model performance
y_pred=svm_model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
accuracy=accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.3385
Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       1.00      0.01      0.02       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.00      0.00      0.00       338
   happiness       0.34      0.32      0.33      1028
        hate       0.45      0.12      0.19       268
        love       0.51      0.35      0.41       762
     neutral       0.32      0.65      0.43      1740
      relief       0.25      0.01      0.01       352
     sadness       0.38      0.15      0.22      1046
    surprise       0.30      0.01      0.03       425
       worry       0.32      0.48      0.39      1666

    accuracy                           0.34      8000
   macro avg       0.30      0.16      0.16      8000
weighted avg       0.34      0.34      0.29      8000

Confusion Matrix:
[[   0    0    0    0    0    1    0    0   10    0    0    0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 However, looking at the precision, recall, and f1-score values, it seems that the model's performance varies across different emotion categories.


In [51]:
new_tweets=['wants to hang out with friends SOON',
              'I HATE CANCER I HATE IT I HATE IT I HATE IT',
              'I need skott right now',
              'I miss Voobys',
              'i m so tired']

In [52]:
preprocessed_new_tweets=[preprocess_text(tweet) for tweet in new_tweets]
X_new=vectorizer.transform(preprocessed_new_tweets)

In [53]:
new_tweet_predictions=svm_model.predict(X_new)

In [54]:
#LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
logreg_model=LogisticRegression()
logreg_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [55]:
y_pred1=logreg_model.predict(X_test)

accuracy=accuracy_score(y_test, y_pred1)
print('AccuracyLOG:', accuracy)
print('Classification ReportLOG:')
print(classification_report(y_test, y_pred1))
print('Confusion MatrixLOG:')
print(confusion_matrix(y_test, y_pred1))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AccuracyLOG: 0.34375
Classification ReportLOG:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       0.33      0.01      0.01       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.04      0.00      0.01       338
   happiness       0.33      0.34      0.33      1028
        hate       0.51      0.13      0.21       268
        love       0.52      0.36      0.43       762
     neutral       0.33      0.60      0.42      1740
      relief       0.42      0.02      0.04       352
     sadness       0.35      0.24      0.28      1046
    surprise       0.30      0.03      0.05       425
       worry       0.33      0.47      0.39      1666

    accuracy                           0.34      8000
   macro avg       0.27      0.17      0.17      8000
weighted avg       0.34      0.34      0.31      8000

Confusion MatrixLOG:
[[   0    0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
#random forest
from imblearn.over_sampling import RandomOverSampler
#apply oversampling to balance the data
oversampler=RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier()
rf_model.fit(X_train_resampled, y_train_resampled)

#predict on the test data
y_pred = rf_model.predict(X_test)

#performance
accuracy=accuracy_score(y_test, y_pred)
classification_report=classification_report(y_test, y_pred)
confusion_matrix=confusion_matrix(y_test, y_pred)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))
#fit and transform the training data
X_train=vectorizer.fit_transform(X_train_text)
X_test=vectorizer.transform(X_test_text)

In [ ]:
from sklearn.model_selection import GridSearchCV
#the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
grid_search=GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train_resampled, y_train_resampled)

In [ ]:
#best model and its performance
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_